# Plan:

- Load the data
- Visualise
- Preprocess
- Dataset definition
- Model definition
- Model training
- Model testing
- Results analysis

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler , LabelEncoder

from torch.utils.data import DataLoader , Dataset
from transformers import BertTokenizer , BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

/Users/pierre/anaconda3/envs/nlpenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the data

In [2]:
train_path = "data/traindata.csv"
val_path = "data/devdata.csv"

df_train = pd.read_csv(train_path, sep='\t', header=0, index_col=False)
df_val = pd.read_csv(val_path, sep='\t', header=0, index_col=False)

print(df_train.head())

   positive    AMBIENCE#GENERAL    seating   18:25  \
0  positive    AMBIENCE#GENERAL  trattoria   25:34   
1  positive        FOOD#QUALITY       food  98:102   
2  negative     SERVICE#GENERAL      STAFF    5:10   
3  positive  FOOD#STYLE_OPTIONS       menu     4:8   
4  positive        FOOD#QUALITY       tuna     4:8   

  short and sweet – seating is great:it's romantic,cozy and private.  
0  This quaint and romantic trattoria is at the t...                  
1  The have over 100 different beers to offer thi...                  
2                        THIS STAFF SHOULD BE FIRED.                  
3  The menu looked great, and the waiter was very...                  
4        The tuna and wasabe potatoes are excellent.                  


# Make the datasets

In [3]:
X_train = df_train.iloc[:, 1:]
y_train = df_train.iloc[:, 0]
print(X_train.shape , y_train.shape)

X_val = df_val.iloc[:, 1:]
y_val = df_val.iloc[:, 0]
print(X_val.shape , y_val.shape)

(1502, 4) (1502,)
(375, 4) (375,)


In [4]:
# check the distribution of the target variable
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))

positive
positive    0.701731
negative    0.259654
neutral     0.038615
Name: proportion, dtype: float64
positive
positive    0.701333
negative    0.261333
neutral     0.037333
Name: proportion, dtype: float64


In [5]:
# Train
aspect_encoder = LabelEncoder()
polarity_encoder = LabelEncoder()
df_train['aspect_category_encoded'] = aspect_encoder.fit_transform(df_train.iloc[:, 1])
y_train_encoded = polarity_encoder.fit_transform(y_train)
X_train_sentences = df_train.iloc[:, -1] # sentence is the last column
print(f"Encoded labels shape: {y_train_encoded.shape}")
print(f"Input sentences shape: {X_train_sentences.shape}")

# Validation
aspect_encoder_val = LabelEncoder()
polarity_encoder_val = LabelEncoder()
df_val['aspect_category_encoded'] = aspect_encoder_val.fit_transform(df_val.iloc[:, 1])
y_val_encoded = polarity_encoder_val.fit_transform(y_val)
X_val_sentences = df_val.iloc[:, -1]
print(f"Encoded labels shape: {y_val_encoded.shape}")
print(f"Input sentences shape: {X_val_sentences.shape}")

Encoded labels shape: (1502,)
Input sentences shape: (1502,)
Encoded labels shape: (375,)
Input sentences shape: (375,)


# Dataset definition

In [6]:
class ReviewDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = str(self.sentences.iloc[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
          sentence,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
        )
        
        return {
          'review_text': sentence,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

batch_size = 32

dataset = ReviewDataset(X_train_sentences, y_train_encoded, tokenizer)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_dataset = ReviewDataset(X_val_sentences, y_val_encoded, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Model definition

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop1 = nn.Dropout(p=0.3)  # First dropout layer
        # Adding a fully connected layer to introduce more capacity to the model
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size // 2)
        self.drop2 = nn.Dropout(p=0.2)  # Second dropout layer
        # Layer normalization
        self.layer_norm = nn.LayerNorm(self.bert.config.hidden_size // 2)
        # Final output layer
        self.out = nn.Linear(self.bert.config.hidden_size // 2, n_classes)
    
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        dropped_output = self.drop1(pooled_output)
        # Applying the fully connected layer after the first dropout
        fc1_output = F.relu(self.fc1(dropped_output))
        dropped_output = self.drop2(fc1_output)
        # Applying layer normalization
        norm_output = self.layer_norm(dropped_output)
        return self.out(norm_output)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentimentClassifier(n_classes=len(polarity_encoder.classes_)).to(device)

optimizer = Adam(model.parameters(), lr=2e-5)

class_proportions = {0: 0.26, 1: 0.04, 2: 0.7}
class_weights = {class_label: (1.0 / proportion) for class_label, proportion in class_proportions.items()}
weight_sum = sum(class_weights.values())
num_classes = len(class_weights)
class_weights = {class_label: (weight / weight_sum) * num_classes for class_label, weight in class_weights.items()}
weights_tensor = torch.tensor(list(class_weights.values()), dtype=torch.float32)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor).to(device)

# Train loop

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return correct_predictions.double() / n_examples, np.mean(losses)

# Evaluation loop

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            loss = loss_fn(outputs, labels)
            
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # metrics
    precision = precision_score(true_labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(true_labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)

    return correct_predictions.double() / n_examples, np.mean(losses), precision, recall, f1

# Training

In [ ]:
def train_model(model, train_data_loader, val_data_loader, loss_fn, optimizer, device, epochs=4):
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 10)

        train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, device, len(train_data_loader.dataset))
        print(f'Train loss {train_loss} accuracy {train_acc}')

        val_acc, val_loss, val_precision, val_recall, val_f1 = eval_model(model, val_data_loader, loss_fn, device, len(val_data_loader.dataset))
        print(f'Val loss {val_loss} accuracy {val_acc}')
        print(f'Precision: {val_precision}, Recall: {val_recall}, F1-score: {val_f1}\n')

In [ ]:
train_model(model, loader, val_loader, loss_fn, optimizer, 'cuda', epochs=10)

# Predictions

In [ ]:
def eval_model_with_predictions(model, data_loader, device):
    
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)


            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

In [ ]:
predictions, true_labels = eval_model_with_predictions(model, val_loader, device)

# Results

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(true_labels, predictions)

fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap="Blues", square=True,
            xticklabels=['Predicted Negative', 'Predicted Neutral', 'Predicted Positive'],
            yticklabels=['Actual Negative', 'Actual Neutral', 'Actual Positive'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()